In [1]:
# load required packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# load required functionality from keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adadelta
from keras.utils import np_utils
from keras import backend as keras_backend
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


Using Theano backend.


ImportError: Traceback (most recent call last):
  File "/home/tom/anaconda3/envs/testJ/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/tom/anaconda3/envs/testJ/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/tom/anaconda3/envs/testJ/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/tom/anaconda3/envs/testJ/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/home/tom/anaconda3/envs/testJ/lib/python3.7/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: libcuda.so.1: cannot open shared object file: No such file or directory


Fai

## MNIST Dataset
The MNIST data set is a large database of handwritten digits (0-9) that is commonly used for training and benchmarking binary classification models.

Before we create any models, we'll start by loading the MNIST data and viewing some sample images.

In [ ]:
from keras.datasets import mnist

# split MNIST data into training and testing sets
# - training: data used to learn the model parameters
# - testing: a separate set that we will use to validate our model performance
(X_train, y_train), (X_test, y_test) = mnist.load_data()
#http://yann.lecun.com/exdb/mnist/ 資料來源

In [ ]:
# check the dimensions of the data
print( X_train.shape ) # 60000筆 28*28

In [ ]:
# select an image
img = X_train[0]

# show the image
plt.imshow(img, cmap='gray')
plt.show()

Notice anything about the images? For example, how many pixels are in each image?
    每張圖都是 28*28，但實際數字的大小好像不到 28*28。反而像 20*20 置中有黑色的邊框
    
Now that we can view the images, let's try also showing the true label alongside the image:

In [ ]:
# select one of the images and its corresponding label
i = 403
img = X_train[i]
label = y_train[i]

# show the image and its label
plt.imshow(img, cmap='gray')
plt.title("True label: %d" % label)
plt.show()

**Logistic regression implemented in Keras**


Before moving onto models, we should pre-process the MNIST data:
1. reshape the MNIST images into 1D arrays (from 2D arrays)
2. normalize the MNIST images
3. convert the MNIST labels

In [ ]:

# reshape the data: (n_samples, 28, 28) => (n_samples, 28*28)
X_train = X_train.reshape(-1, 28 * 28)
X_test = X_test.reshape(-1, 28 * 28)

# convert data type and normalize the values (8-bit = 256 = 0...255)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# convert the class labels to 10-dimensional class arrays:
# 這邊就是上課說到 0~9 有大小差別的問題，所以轉成 bag of words
# - before: y_train = (n_samples, )
# - after: Y_train = (n_samples, 10)
#
Y_train = np_utils.to_categorical(y_train, 10)# keras 用 numpy 寫成的 function 
Y_test = np_utils.to_categorical(y_test, 10)# 詳情看 https://keras.io/utils/

#y_Train = np.eye(10)[y_train] #這樣寫有同樣的效果 identity matrix 第 y_train row
#print(y_Train[0])

In [ ]:

print(y_train[0], Y_train[0])

In [ ]:
# create the model
model = Sequential()
model.add(Dense(10, input_dim=28*28, activation='softmax'))

# compile the model
#model.compile(loss='categorical_crossentropy',
#              optimizer='sgd',
#              metrics=['accuracy'])            
model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])            


# train the model on data
model.fit(X_train, Y_train,
          batch_size=128, # how many images to look at together
          nb_epoch=20,       # how many epochs to run before stopping
          verbose=1,
          validation_data=(X_test, Y_test)
         )

# test the trained model on the testing set
score = model.evaluate(X_test, Y_test, verbose=0) 

print('Test score:', score[0]) 
print('Test accuracy:', score[1])

In [ ]:
# select an image
i = 200
img = X_test[i].reshape(28, 28)

# prepare the image to be used in the model
x = img.reshape(-1, 28*28)

# get the model output
model_label = np.argmax( model.predict(x) )

# get the true label
true_label = np.argmax(Y_test[i])

plt.imshow(img, cmap='gray')
plt.title('True label: {0}, Model label: {1}'.format(true_label, model_label))
plt.show()

**Multilayer Perceptron (MLP)**

Next we'll try out a "classic" neural network model (i.e. not a deep neural network).

A Multilayer Perceptron (aka an Artificial Neural Network) is made up of an input layer, one or more hidden layers, and an output layer

In [ ]:
# create the MLP model
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(28*28,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

# print a summary of the model topology
#model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

model.fit(X_train, Y_train,
          batch_size=128,
          epochs=2,
          verbose=1,
          validation_data=(X_test, Y_test))

score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

**Discussion**
**Question 1:** How did the accuracy of the MLP model compare to the Logistic Regression model?

**Question 2:** Can you identify any other differences between the MLP and Logistic Regression models? *Hint:* Was there a difference in the code complexity and/or training time?

**Convolutional Neural Network (CNN)**
Now let's try out first deep neural network: a Convolutional Neural Network (CNN).

The CNN is made up of a few core layer types, which get stacked on top of each other:
- convolutional layers (2D)
- max pooling layers (2D)
- fully connected (aka densely connected) layers (same type as in the MLP model)


First, we need to perform pre-processing on the MNIST data, but with a slight tweak from the previous examples:

In [ ]:
# load the data again (to be safe)
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape the data based on what backend is in use (TensorFlow or Thean)
if keras_backend.image_data_format() == 'channels_first':
    X_train = X_train.reshape(-1, 1, 28, 28)
    X_test = X_test.reshape(-1, 1, 28, 28)
else:
    X_train = X_train.reshape(-1, 28, 28, 1)
    X_test = X_test.reshape(-1, 28, 28, 1)

# convert data type and normalize the values (8-bit = 256 = 0...255)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# convert the class labels to 10-dimensional class arrays:
# - before: y_train = (n_samples, )
# - after: Y_train = (n_samples, 10)
#
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [ ]:
# create the CNN model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)
                ))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, Y_train,
          batch_size=128,
          epochs=2,
          verbose=1,
          validation_data=(X_test, Y_test))

score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

**Discussion**
**Question 1:** How did the accuracy of the CNN compare to the MLP and Logistic Regression models?

**Question 2:** Can you identify any other differences between the three models?